In [1]:
import peforth

In [3]:
import pandas as pd
import tensorflow as tf

In [4]:
CATEGORICAL_COLUMNS = ["Name", "Sex", "Embarked", "Cabin"]
CONTINUOUS_COLUMNS = ["Age", "SibSp", "Parch", "Fare", "PassengerId", "Pclass"]
SURVIVED_COLUMN = "Survived"

In [7]:
sex = tf.feature_column.categorical_column_with_vocabulary_list("Sex",["female", "male"])
embarked = tf.feature_column.categorical_column_with_vocabulary_list("Embarked",["C","S","Q"])

In [8]:
  cabin = tf.feature_column.categorical_column_with_hash_bucket(
      "Cabin", hash_bucket_size=1000)
  name = tf.feature_column.categorical_column_with_hash_bucket(
      "Name", hash_bucket_size=1000)

In [9]:
  # Continuous columns
  age = tf.feature_column.numeric_column("Age")
  passenger_id = tf.feature_column.numeric_column("PassengerId")
  sib_sp = tf.feature_column.numeric_column("SibSp")
  parch = tf.feature_column.numeric_column("Parch")
  fare = tf.feature_column.numeric_column("Fare")
  p_class = tf.feature_column.numeric_column("Pclass")

In [10]:
  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(age,
                                                    boundaries=[
                                                        5, 18, 25, 30, 35, 40,
                                                        45, 50, 55, 65
                                                    ])

In [29]:
  # Wide columns and deep columns.
  wide_columns = [sex, embarked, cabin, name, age_buckets,
                  tf.feature_column.crossed_column(
                      [age_buckets, sex],
                      hash_bucket_size=int(1e6)),
                  tf.feature_column.crossed_column([embarked, 'name'],
                                                   hash_bucket_size=int(1e4))]

In [18]:
tf.feature_column.crossed_column([age_buckets, sex],hash_bucket_size=int(1e6))

_CrossedColumn(keys=(_BucketizedColumn(source_column=_NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(5, 18, 25, 30, 35, 40, 45, 50, 55, 65)), _VocabularyListCategoricalColumn(key='Sex', vocabulary_list=('female', 'male'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), hash_bucket_size=1000000, hash_key=None)

In [23]:
tf.feature_column.crossed_column([embarked, 'name'],hash_bucket_size=int(1e4))
# 這行出錯 ValueError: categorical_column_with_hash_bucket is not supported for crossing. 
# Hashing before crossing will increase probability of collision. Instead, use the feature name as a string.
# [ ] 從 2018 新 demo 裡去找答案範例 . . . 看到 GitHub\models\official\wide_deep\census_dataset.py 的例子，
#     改用 'Name' 或 'name' 都會好。


_CrossedColumn(keys=(_VocabularyListCategoricalColumn(key='Embarked', vocabulary_list=('C', 'S', 'Q'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'name'), hash_bucket_size=10000, hash_key=None)

In [25]:
  deep_columns = [
      tf.feature_column.embedding_column(sex, dimension=8),
      tf.feature_column.embedding_column(embarked, dimension=8),
      tf.feature_column.embedding_column(cabin, dimension=8),
      tf.feature_column.embedding_column(name, dimension=8),
      age,
      passenger_id,
      sib_sp,
      parch,
      fare,
      p_class
  ]

In [30]:
  #return 
tf.estimator.DNNLinearCombinedClassifier(
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=[100, 50])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hcche\\AppData\\Local\\Temp\\tmp206bhb4g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016BD8AD1940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### 以上已經解決了 def build_estimator(model_dir) 的所有問題

In [36]:
def build_estimator(model_dir):
  """Build an estimator."""
  # Categorical columns
  sex = tf.feature_column.categorical_column_with_vocabulary_list("Sex",["female", "male"])
  embarked = tf.feature_column.categorical_column_with_vocabulary_list("Embarked",["C","S","Q"])

  cabin = tf.feature_column.categorical_column_with_hash_bucket(
      "Cabin", hash_bucket_size=1000)
  name = tf.feature_column.categorical_column_with_hash_bucket(
      "Name", hash_bucket_size=1000)

  # Continuous columns
  age = tf.feature_column.numeric_column("Age")
  passenger_id = tf.feature_column.numeric_column("PassengerId")
  sib_sp = tf.feature_column.numeric_column("SibSp")
  parch = tf.feature_column.numeric_column("Parch")
  fare = tf.feature_column.numeric_column("Fare")
  p_class = tf.feature_column.numeric_column("Pclass")

  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(age,
                                                    boundaries=[
                                                        5, 18, 25, 30, 35, 40,
                                                        45, 50, 55, 65
                                                    ])
# Wide columns and deep columns.
  wide_columns = [sex, embarked, cabin, name, age_buckets,
                  tf.feature_column.crossed_column(
                      [age_buckets, sex],
                      hash_bucket_size=int(1e6)),
                  tf.feature_column.crossed_column([embarked, 'name'],
                                                   hash_bucket_size=int(1e4))]    
  deep_columns = [
      tf.feature_column.embedding_column(sex, dimension=8),
      tf.feature_column.embedding_column(embarked, dimension=8),
      tf.feature_column.embedding_column(cabin, dimension=8),
      tf.feature_column.embedding_column(name, dimension=8),
      age,
      passenger_id,
      sib_sp,
      parch,
      fare,
      p_class
  ]

  return tf.estimator.DNNLinearCombinedClassifier(
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=[100, 50])

def input_fn(df, train=False):
  """Input builder function."""
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
  categorical_cols = {k: tf.SparseTensor(
    indices=[[i, 0] for i in range(df[k].size)],
    values=df[k].values,
    shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
  feature_cols = dict(continuous_cols)
  feature_cols.update(categorical_cols)
  # Converts the label column into a constant Tensor.
  if train:
    label = tf.constant(df[SURVIVED_COLUMN].values)
      # Returns the feature columns and the label.
    return feature_cols, label
  else:
    return feature_cols


def train_and_eval():
  """Train and evaluate the model."""
  df_train = pd.read_csv(
      tf.gfile.Open("./train.csv"),
      skipinitialspace=True)
  df_test = pd.read_csv(
      tf.gfile.Open("./test.csv"),
      skipinitialspace=True)

  model_dir = "./models"
  print("model directory = %s" % model_dir)

  m = build_estimator(model_dir)
  # m.fit(input_fn=lambda: input_fn(df_train, True), steps=200)
  print (m.predict(input_fn=lambda: input_fn(df_test)))
  results = m.evaluate(input_fn=lambda: input_fn(df_train, True), steps=1)
  for key in sorted(results):
    print("%s: %s" % (key, results[key]))

def main(_):
  train_and_eval()


if __name__ == "__main__":
  tf.app.run()

model directory = ./models
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hcche\\AppData\\Local\\Temp\\tmpbiik21pi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016BD8BC78D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
<generator object Estimator.predict at 0x0000016BD8BBF728>


ValueError: Could not find trained model in model_dir: C:\Users\hcche\AppData\Local\Temp\tmpbiik21pi.

In [ ]:
main()

In [2]:
%%f if playing in jupyter notebook 
\ Now we redefine the 'unknown' command that does nothing at default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: add-path ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code # print(nexttoken('\n')) end-code // print the # leading comment

reDef unknown


In [35]:
%f *debug*


__name__ tib.
__name__ tib. \ ==> __main__ (<class 'str'>)
exit
